# 19. Dynamic Attributes and Properties

파이썬에서는 데이터 속성과 메서드를 통틀어 **속성**이라고 한다. 메서드는 단지 **호출할 수 있는 속성**일 뿐이다. 데이터 속성과 메서드 외에도 프로퍼티(속성)를 정의할 수 있다. 프로퍼티를 사용하면 클래스 인터페이스를 변경하지 않고도 공개 데이터 속성을 **접근자 메서드**로 대체할 수 있다. 이것은 **통일된 접근 원칙**에도 부합한다.

**통일된 접근 원칙** : 모듈이 제공하는 모든 서비스는 토일된 표기법을 잉요해서 접근할 수 있어야 한다. 통일된 표기법은 저장소를 이용해서 구현하거나 계산을 통해 구현하는 경우에도 모두 동일하게 적용된다. (버트랜드 메이어의 『Object-Oriented Software Construction』에서 발췌)

## 동적 속성을 이용한 데이터 랭글링

In [1]:
from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'

def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)
        with urlopen(URL) as remote, open(JSON, 'wb') as local:
            local.write(remote.read())
            
    with open(JSON) as fp:
        return json.load(fp)

위 코드로 데이터에 들어 있는 어떠한 필드도 조사 가능하다.

In [2]:
feed = load()
sorted(feed['Schedule'].keys())

C:\Program Files (x86)\Anaconda\Ana_conda\lib\site-packages\ipykernel_launcher.py:12: UserWarning: downloading http://www.oreilly.com/pub/sc/osconfeed to data/osconfeed.json
  if sys.path[0] == '':


FileNotFoundError: [Errno 2] No such file or directory: 'data/osconfeed.json'

왜 에러가 나는 것인지는 잘 모르겠다. 사이트 내에 파일이 없어진 듯하다...

## 속성을 검증하기 위해 프로퍼티 사용하기

책에 나온 예시로, 유기농산물을 판매하는 상점을 위한 앱을 생각해 보자.  
이 시스템에서 각각의 주문에는 일련의 품목명이 들어가며, 각 품목명은 아래의 클래스로 표현된다.

In [9]:
class LineItem:

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

하지만 이것만으로는, 아래와 같이 음수 값을 입력하는 등의 상황에서 문제가 생길 수 있다! (실제로 아마존 사이트에서 이러한 일이 일어났다고 한다...)

In [10]:
raisins = LineItem('Golden raisins', 10, 6.95)
print(raisins.subtotal())
raisins.weight = -20
print(raisins.subtotal())

69.5
-139.0


이는 아래와 같이 데이터 속성을 프로퍼티로 변경함으로써 해결할 수 있다.

In [22]:
class Lineitem:
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
            return self.weight * self.price

    @property
    def weight(self):
            return self.__weight

    @weight.setter
    def weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('value must be > 0')

In [23]:
walnuts = Lineitem('walnuts', -10, 6.95)
print(raisins.subtotal())

ValueError: value must be > 0